In [1]:
base_dir = "/home3/skaasyap/willett"

In [2]:
import os
from glob import glob
from pathlib import Path
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
# os.environ["CUDA_VISIBLE_DEVICES"]=""

import numpy as np
from omegaconf import OmegaConf
from neural_decoder.bit import BiT_Phoneme
import neural_decoder.lm_utils as utils

/home3/skaasyap/miniconda3/envs/willett/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
lmDir = base_dir+'/lm/languageModel'
ngramDecoder = utils.build_lm_decoder(
    lmDir,
    acoustic_scale=0.8, #1.2
    nbest=1,
    beam=18
)

I0426 22:31:59.210306 2766702 brain_speech_decoder.h:52] Reading fst /home3/skaasyap/willett/lm/languageModel/TLG.fst
I0426 22:35:15.268976 2766702 brain_speech_decoder.h:81] Reading symbol table /home3/skaasyap/willett/lm/languageModel/words.txt


In [4]:
# lmDir = base_dir+'/lm_5gram/speech_5gram/lang_test'
# ngramDecoder_5gram = utils.build_lm_decoder(
#     lmDir,
#     acoustic_scale=0.5,
#     nbest=100,
#     beam=18
# )

In [6]:
import pickle 
from neural_decoder.model import GRUDecoder
import torch
def loadModel(modelDir, nInputLayers=24, device="cuda:3"):
    modelWeightPath = modelDir + "/modelWeights"
    with open(modelDir + "/args", "rb") as handle:
        args = pickle.load(handle)

    print(args)

    model = GRUDecoder(
        neural_dim=args["nInputFeatures"],
        n_classes=args["nClasses"],
        hidden_dim=args["nUnits"],
        layer_dim=args["nLayers"],
        nDays=nInputLayers,
        dropout=args["dropout"],
        device=device,
        strideLen=args["strideLen"],
        kernelLen=args["kernelLen"],
        gaussianSmoothWidth=args["gaussianSmoothWidth"],
        bidirectional=args["bidirectional"],
    ).to(device)

    model.load_state_dict(torch.load(modelWeightPath, map_location=device))
    return model


In [115]:
args = {}
def loadTransformerModel(modelDir, device="cuda:3"):
    modelWeightPath = modelDir + "/modelWeights"
    with open(modelDir + "/args", "rb") as handle:
        args = pickle.load(handle)

    print(args)

    model = BiT_Phoneme(
        patch_size=args['patch_size'],
        dim=args['dim'],
        dim_head=args['dim_head'], 
        nClasses=args['nClasses'],
        depth=args['depth'],
        heads=args['heads'],
        mlp_dim_ratio=args['mlp_dim_ratio'],
        dropout=args['dropout'],
        input_dropout=args['input_dropout'],
        look_ahead=0,
        gaussianSmoothWidth=args['gaussianSmoothWidth'],
        T5_style_pos=args['T5_style_pos'], 
        max_mask_pct=args['max_mask_pct'], 
        num_masks=args['num_masks'], 
        ).to(device)

    model.load_state_dict(torch.load(modelWeightPath, map_location=device))
    return model

In [141]:
model = loadTransformerModel("/home3/skaasyap/willett/outputs/neurips_model_seed_1")

{'outputDir': '/home3/skaasyap/willett/outputs/neurips_model_seed_1', 'modelName': 'neurips_model_seed_1', 'datasetPath': '/home3/skaasyap/willett/data_log_both', 'patch_size': (5, 256), 'dim': 384, 'depth': 7, 'heads': 6, 'mlp_dim_ratio': 4, 'dim_head': 64, 'dropout': 0.35, 'input_dropout': 0.2, 'whiteNoiseSD': 0.2, 'gaussianSmoothWidth': 2.0, 'constantOffsetSD': 0.05, 'nDays': 24, 'nClasses': 40, 'batchSize': 64, 'l2_decay': 1e-05, 'AdamW': True, 'learning_scheduler': 'multistep', 'lrStart': 0.001, 'lrEnd': 0.001, 'milestones': [400], 'gamma': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'look_ahead': 0, 'extra_notes': '', 'device': 'cuda:0', 'seed': 1, 'T5_style_pos': True, 'n_epochs': 2000, 'load_pretrained_mae': '', 'max_mask_pct': 0.075, 'num_masks': 20, 'dist_dict_path': '/home3/skaasyap/willett/outputs/dist_dict.pt'}


In [89]:
model = loadModel("/home3/skaasyap/willett/outputs/neurips_model_seed_0")

{'outputDir': '/home3/skaasyap/willett/outputs/neurips_model_seed_0', 'modelName': 'neurips_model_seed_0', 'datasetPath': '/home3/skaasyap/willett/data_log_both', 'patch_size': (5, 256), 'dim': 384, 'depth': 7, 'heads': 6, 'mlp_dim_ratio': 4, 'dim_head': 64, 'dropout': 0.35, 'input_dropout': 0.2, 'whiteNoiseSD': 0.2, 'gaussianSmoothWidth': 2.0, 'constantOffsetSD': 0.05, 'nDays': 24, 'nClasses': 40, 'batchSize': 64, 'l2_decay': 1e-05, 'AdamW': True, 'learning_scheduler': 'multistep', 'lrStart': 0.001, 'lrEnd': 0.001, 'milestones': [400], 'gamma': 0.1, 'beta1': 0.9, 'beta2': 0.999, 'look_ahead': 0, 'extra_notes': '', 'device': 'cuda:2', 'seed': 0, 'T5_style_pos': True, 'n_epochs': 2000, 'load_pretrained_mae': '', 'max_mask_pct': 0.075, 'num_masks': 20, 'dist_dict_path': '/home3/skaasyap/willett/outputs/dist_dict.pt'}


KeyError: 'nInputFeatures'

In [142]:
from torch.utils.data import DataLoader

from neural_decoder.model import GRUDecoder
from neural_decoder.dataset import SpeechDataset
from torch.nn.utils.rnn import pad_sequence

def getDatasetLoaders(
    datasetName,
    batchSize,
):
    with open(datasetName, "rb") as handle:
        loadedData = pickle.load(handle)

    def _padding(batch):
        X, y, X_lens, y_lens, days = zip(*batch)
        X_padded = pad_sequence(X, batch_first=True, padding_value=0)
        y_padded = pad_sequence(y, batch_first=True, padding_value=0)

        return (
            X_padded,
            y_padded,
            torch.stack(X_lens),
            torch.stack(y_lens),
            torch.stack(days),
        )

    train_ds = SpeechDataset(loadedData["train"], transform=None)
    test_ds = SpeechDataset(loadedData["test"])

    train_loader = DataLoader(
        train_ds,
        batch_size=batchSize,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        collate_fn=_padding,
    )
    test_loader = DataLoader(
        test_ds,
        batch_size=batchSize,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=_padding,
    )

    return train_loader, test_loader, loadedData


In [143]:
trainLoader, testLoader, loadedData = getDatasetLoaders(
    "/home3/skaasyap/willett/data_log_both",
    8,
)

In [144]:
device = 'cuda:3'

In [145]:
print(model)

BiT_Phoneme(
  (inputLayerNonlinearity): Softsign()
  (to_patch_embedding): Sequential(
    (0): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=5, p2=256)
    (1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (2): Linear(in_features=1280, out_features=384, bias=True)
    (3): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  )
  (to_patch): Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=5, p2=256)
  (patch_to_emb): Sequential(
    (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (1): Linear(in_features=1280, out_features=384, bias=True)
    (2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  )
  (gaussianSmoother): GaussianSmoothing()
  (dropout): Dropout(p=0.2, inplace=False)
  (transformer): Transformer(
    (norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0): ModuleList(
        (0): Attention(
          (norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (at

In [146]:
print(device)

cuda:3


In [105]:
# loss_ctc = torch.nn.CTCLoss(blank=0, reduction="mean", zero_infinity=True)
# from edit_distance import SequenceMatcher

# all_logits = []
# trial_lengths = []
# decoded_seqs = []
# true_seqs = []

# with torch.no_grad():
#     model.eval()
#     allLoss = []
#     total_edit_distance = 0
#     total_seq_length = 0
#     counter = 0
#     minicounter = 0
#     for X, y, X_len, y_len, testDayIdx in testLoader:
#         # if(counter == 10):
#         #     break
#         counter += 1
#         X, y, X_len, y_len, testDayIdx = (
#             X.to(device),
#             y.to(device),
#             X_len.to(device),
#             y_len.to(device),
#             testDayIdx.to(device),
#         )

#         pred = model.forward(X, X_len, testDayIdx)
#         all_logits.append(pred)
#         loss = loss_ctc(
#             torch.permute(pred.log_softmax(2), [1, 0, 2]),
#             y,
#             ((X_len - model.kernelLen) / model.strideLen).to(torch.int32),
#             y_len,
#         )
#         loss = torch.sum(loss)
#         allLoss.append(loss.cpu().detach().numpy())

#         adjustedLens = ((X_len - model.kernelLen) / model.strideLen).to(
#             torch.int32
#         )
#         trial_lengths.append(adjustedLens)
#         for iterIdx in range(pred.shape[0]):
#             decodedSeq = torch.argmax(
#                 torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),
#                 dim=-1,
#             )  # [num_seq,]
#             decodedSeq = torch.unique_consecutive(decodedSeq, dim=-1)
#             decodedSeq = decodedSeq.cpu().detach().numpy()
#             decodedSeq = np.array([i for i in decodedSeq if i != 0])
#             # print("decodedSeq", decodedSeq.shape)
#             trueSeq = np.array(
#                 y[iterIdx][0 : y_len[iterIdx]].cpu().detach()
#             )

#             matcher = SequenceMatcher(
#                 a=trueSeq.tolist(), b=decodedSeq.tolist()
#             )
#             print(minicounter)
#             minicounter += 1
#             print(decodedSeq.tolist())
#             print(trueSeq.tolist())
#             print('-'*50)
#             decoded_seqs.append(decodedSeq)
#             true_seqs.append(trueSeq)
#             total_edit_distance += matcher.distance()
#             total_seq_length += len(trueSeq)

#     avgDayLoss = np.sum(allLoss) / len(testLoader)
#     cer = total_edit_distance / total_seq_length

#     print(
#         f"ctc loss: {avgDayLoss:>7f}, cer: {cer:>7f}"
#     )


/tmp/ipykernel_2766702/3262995781.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


0
[16, 17, 21, 40, 9, 31, 3, 38, 40, 29, 18, 40, 17, 38, 40, 22, 2, 20, 15, 5]
[32, 18, 1, 20, 28, 3, 29, 18, 40, 28, 18, 20, 3, 23, 29, 17, 9, 12, 9, 40]
--------------------------------------------------
1
[29, 34, 29, 3, 10, 13, 29, 18, 18, 40, 17, 31, 40, 31, 13, 40, 7, 28]
[28, 17, 8, 40, 27, 12, 8, 3, 29, 31, 40, 29, 11, 35, 28, 3, 21, 40, 29, 6, 23, 9, 40, 21, 17, 32, 3, 15, 28, 2, 14, 29, 40]
--------------------------------------------------
2
[16, 6, 40, 9, 18, 3, 29, 40, 29, 6, 9, 23, 18, 40]
[29, 25, 40, 28, 34, 21, 38, 40, 36, 18, 40, 22, 13, 9, 40, 17, 23, 40, 3, 23, 3, 7, 2, 30, 31, 40, 20, 3, 21, 34, 39, 3, 23, 40]
--------------------------------------------------
3
[37, 29, 16, 12, 40, 6, 21, 33, 20, 10, 3, 34, 13, 31, 3, 24, 40, 16, 17, 29, 31, 40, 3, 40]
[21, 4, 28, 18, 38, 40, 20, 1, 29, 31, 34, 22, 40, 23, 18, 9, 3, 9, 40, 7, 21, 2, 20, 40, 15, 21, 3, 35, 38, 40, 31, 34, 40, 7, 18, 40, 20, 3, 22, 27, 21, 18, 31, 21, 18, 40, 11, 21, 3, 15, 3, 23, 31, 40]
----------

In [155]:
loss_ctc = torch.nn.CTCLoss(blank=0, reduction="mean", zero_infinity=True)
from edit_distance import SequenceMatcher
args['device'] = 'cuda:3'

transformer_logits = []
trial_lengths = []
with torch.no_grad():
    
    model.eval()
    allLoss = []
    total_edit_distance = 0
    total_seq_length = 0
    
    for X, y, X_len, y_len, testDayIdx in testLoader:
        

        X, y, X_len, y_len, testDayIdx = (
            X.to(args["device"]),
            y.to(args["device"]),
            X_len.to(args["device"]),
            y_len.to(args["device"]),
            testDayIdx.to(args["device"]),
        )

        pred = model.forward(X, X_len, testDayIdx)
        transformer_logits.append(pred)
        
        adjustedLens = model.compute_length(X_len)
        trial_lengths.append(adjustedLens)
        
        loss = loss_ctc(
            torch.permute(pred.log_softmax(2), [1, 0, 2]),
            y,
            adjustedLens,
            y_len,
        )
        
        allLoss.append(loss.cpu().detach().numpy())

        for iterIdx in range(pred.shape[0]):
            decodedSeq = torch.argmax(
                torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),
                dim=-1,
            )  # [num_seq,]
            decodedSeq = torch.unique_consecutive(decodedSeq, dim=-1)
            decodedSeq = decodedSeq.cpu().detach().numpy()
            decodedSeq = np.array([i for i in decodedSeq if i != 0])

            trueSeq = np.array(
                y[iterIdx][0 : y_len[iterIdx]].cpu().detach()
            )

            matcher = SequenceMatcher(
                a=trueSeq.tolist(), b=decodedSeq.tolist()
            )
            total_edit_distance += matcher.distance()
            total_seq_length += len(trueSeq)

    avgDayLoss = np.mean(allLoss)
    cer = total_edit_distance / total_seq_length

/tmp/ipykernel_2766702/2415073933.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(pred[iterIdx, 0 : adjustedLens[iterIdx], :]),


In [156]:
print(cer)

0.1382526515620486


In [74]:
PHONE_DEF = [
    'AA', 'AE', 'AH', 'AO', 'AW',
    'AY', 'B',  'CH', 'D', 'DH',
    'EH', 'ER', 'EY', 'F', 'G',
    'HH', 'IH', 'IY', 'JH', 'K',
    'L', 'M', 'N', 'NG', 'OW',
    'OY', 'P', 'R', 'S', 'SH',
    'T', 'TH', 'UH', 'UW', 'V',
    'W', 'Y', 'Z', 'ZH'
]
PHONE_DEF_SIL = PHONE_DEF + ['SIL']

In [75]:
phonemes_decoded = []
phonemes_truth = []

In [76]:
for i in range(len(decoded_seqs)):
    cur = []
    for j in range(len(decoded_seqs[i])):
        cur.append(PHONE_DEF_SIL[decoded_seqs[i][j] - 1])
    phonemes_decoded.append(cur)

In [77]:
for i in range(len(true_seqs)):
    cur = []
    for j in range(len(true_seqs[i])):
        cur.append(PHONE_DEF_SIL[true_seqs[i][j] - 1])
    phonemes_truth.append(cur)

In [78]:
for i in range(0, 50):
    print(phonemes_truth[i])
    print(phonemes_decoded[i])
    print(s[i])
    print()
    print('-'*50)

['TH', 'IY', 'AA', 'K', 'R', 'AH', 'S', 'IY', 'SIL', 'R', 'IY', 'K', 'AH', 'N', 'S', 'IH', 'D', 'ER', 'D', 'SIL']
['DH', 'EY', 'SIL', 'AA', 'K', 'AH', 'S', 'SIL', 'R', 'IH', 'AE', 'N', 'IH', 'D', 'SIL']
theocracy reconsidered

--------------------------------------------------
['R', 'IH', 'CH', 'SIL', 'P', 'ER', 'CH', 'AH', 'S', 'T', 'SIL', 'S', 'EH', 'V', 'R', 'AH', 'L', 'SIL', 'S', 'AY', 'N', 'D', 'SIL', 'L', 'IH', 'TH', 'AH', 'G', 'R', 'AE', 'F', 'S', 'SIL']
['R', 'AE', 'K', 'T', 'S', 'SIL', 'P', 'OW', 'JH', 'CH', 'AH', 'T', 'SIL', 'EH', 'V', 'R', 'AH', 'L', 'SIL', 'K', 'AE', 'N', 'D', 'SIL', 'L', 'IH', 'S', 'T', 'AH', 'G', 'AY', 'F', 'SIL']
rich purchased several signed lithographs

--------------------------------------------------
['S', 'OW', 'SIL', 'R', 'UW', 'L', 'Z', 'SIL', 'W', 'IY', 'SIL', 'M', 'EY', 'D', 'SIL', 'IH', 'N', 'SIL', 'AH', 'N', 'AH', 'B', 'AE', 'SH', 'T', 'SIL', 'K', 'AH', 'L', 'UW', 'ZH', 'AH', 'N', 'SIL']
['DH', 'OW', 'SIL', 'R', 'UW', 'Z', 'SIL', 'B', 'IY', '

In [1]:
print(s[0])

NameError: name 's' is not defined

In [129]:
print(len(transformer_logits))

110


In [119]:
import torch
from torch.nn.utils.rnn import pad_sequence
all_logits = [l.cpu().numpy().astype('float32') for l in all_logits]
trial_lengths = [l.cpu().numpy().astype('int') for l in trial_lengths]

In [157]:
transformer_logits = [l.cpu().numpy().astype('float32') for l in transformer_logits]
trial_lengths = [l.cpu().numpy().astype('int') for l in trial_lengths]

In [158]:
print(len(transformer_logits))

110


In [159]:
transformer_logits = [l for batch in transformer_logits for l in list(batch)]
trial_lengths = [l for batch in trial_lengths for l in list(batch)]

In [160]:
print(len(transformer_logits))
print(len(trial_lengths))

880
880


In [125]:
all_logits = [l for batch in all_logits for l in list(batch)]
trial_lengths = [l for batch in trial_lengths for l in list(batch)]
# maxLogitLength = max([l.shape[0] for l in all_logits])
# all_logits = [np.pad(l, [[0, maxLogitLength-l.shape[0]], [0, 0]]) for l in all_logits]
# all_logits = np.stack(all_logits, axis=0)

In [81]:
print(len(trial_lengths))

880


In [82]:
print("hi")

hi


In [126]:
acoustic_scale = 0.8
blank_penalty = np.log(2)
llm_weight = 0.5

In [84]:
print(len(all_logits))

880


In [161]:
acoustic_scale = 0.8
blank_penalty = np.log(2)
llm_weight = 0.5

llm_outputs = []
# Generate nbest outputs from 5gram LM
nbest_outputs = []
for j in range(len(transformer_logits)):
    logits = transformer_logits[j][:trial_lengths[j], :]
    # print(logits[:, 1:].shape, logits[:, 0:1].shape)
    logits = np.concatenate(
        [logits[:, 1:], logits[:, 0:1]], axis=-1
    )  # Blank is last token
    logits = utils.rearrange_speech_logits(logits[None, :, :], has_sil=True)
    nbest = utils.lm_decode(
        ngramDecoder,
        logits[0],
        blankPenalty=blank_penalty,
        returnNBest=False,
        rescore=False,
    )
    nbest_outputs.append(nbest)
    print(nbest)
    llm_outputs.append(nbest)

 cactus rented
 each purchase evel signed lithographs
 the rules we made in and mass collusion
 cries costume need black gloves to be completely allen
 the tooth fey rot to hum one rider's tooth fell out
 that singing pepper for was cause but could reposition
 before release exam review every formula
 garda for their son have pork
 the word means it won't boil away easily nothing else
 would a booth feather in a man height make him happy all day
 he talked about on authentic stolen u
 both young rise early every morning
 with this new loyal salad can roll
 many tribes have an upbeat edited
 and it's not a charge by a choices of the pace who made the decision
 we seemed to be earning the population explosion right in or on backwards
 summons desperate measures and also to incite moves for the future
 the axles will be right void if if you think them usually carefully first
 she got dressed fresh and times less energy making cooling decisions
 animate aptitude intents you diplomas
 to so

In [7]:
for i in range(len(s)):
    if('find something that works' in s[i].lower()):
        print(s[i])
        

It's really hard to find something that works.


In [17]:
import pickle
with open('my_list.pkl', 'rb') as f:
    s = pickle.load(f)

In [173]:
for i in range(80, 120):
    print(i, ":", s[i], "preds:", llm_outputs[i])

80 : it's really hard to find something that works preds: it's really hard to find something that works
81 : and i loved ths shining preds: and i loved the shining
82 : i do have a friend that ran preds: i do have a friend that run
83 : just way in the back preds: dust way in the back
84 : actually it makes sense to a certain extent preds: actually it makes sense to a certain extent
85 : assure the government that he was employed preds: assure the government that he was employed
86 : they feel good about what they're doing preds: they feel good about what they're doing
87 : besides its an invasion of personal privacy preds: besides it's an invasion of personal privacy
88 : so you travel a lot preds: so you travel a lot
89 : so we're going to wait preds: so we're going to write
90 : any kind of a good environment to live in preds: any kind of a good environment to live in
91 : the best of android preds: the best of android
92 : i don't know what you think about the point preds: i don't 

In [166]:
for i in range(len(s)):
    s[i] = s[i].strip()
    llm_outputs[i] = llm_outputs[i].strip()

In [171]:
pri

theocracy reconsidered


In [167]:
for i in range(len(s)):
    s[i] = convert_sentence(s[i])

In [168]:
cer, wer = utils._cer_and_wer(llm_outputs, s, outputType='speech_sil', returnCI=True)

In [169]:
print(cer, wer)

(0.11273744234191697, 0.10415047596387642, 0.12164697107997179) (0.16730314602655028, 0.1548932539111239, 0.1801721467214425)


In [23]:
cer, wer = utils._cer_and_wer(llm_outputs, s, outputType='speech_sil')

In [24]:
print(cer, wer)

0.22966608084358522 0.6250227314057101


In [33]:
def wer(r, h):
    """
    Calculation of WER with Levenshtein distance.
    Works only for iterables up to 254 elements (uint8).
    O(nm) time ans space complexity.
    Parameters
    ----------
    r : list
    h : list
    Returns
    -------
    int
    Examples
    --------
    >>> wer("who is there".split(), "is there".split())
    1
    >>> wer("who is there".split(), "".split())
    3
    >>> wer("".split(), "who is there".split())
    3
    """
    # initialisation
    import numpy
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]

In [38]:
print(llm_outputs)

[]


In [42]:
print(utils._cer_and_wer(llm_outputs, s, outputType='speech_sil'))

NameError: name 'rnnEval' is not defined

In [35]:
print(llm_outputs[0])

 the artist recounted


In [36]:
print(s[0])

Theocracy reconsidered.


In [34]:
trueWords = s[0].split(" ")
decWords = llm_outputs[0].split(" ")
nWordErr = wer(trueWords, decWords)
print(nWordErr/len(trueWords))


2.0


In [45]:
print(trueWords, decWords)

['Theocracy', 'reconsidered.'] ['', 'the', 'artist', 'recounted']


In [51]:
for i in range(len(s)):
    print(s[i])
    print(llm_outputs[i])
    print('-'*50)

Theocracy reconsidered.
the artist recounted
--------------------------------------------------
Rich purchased several signed lithographs.
gas purchase ever and lithograph
--------------------------------------------------
So rules we made, in unabashed collusion.
the rules we made in unabashed collusion
--------------------------------------------------
Lori's costume needed black gloves to be completely elegant.
wars taub did make love to be completely allen
--------------------------------------------------
The tooth fairy forgot to come when Roger's tooth fell out.
the tooth fairy feet to come one way to fill out
--------------------------------------------------
That stinging vapor was caused by chloride vaporization.
that singing if i was called a torrid deeper listen
--------------------------------------------------
Before Thursday's exam, review every formula.
before thursday's game was ever formally
--------------------------------------------------
Wildfire near Sunshine for

In [52]:
import jiwer

for i in range(len(s)): 
    wer = jiwer.wer(s[i], llm_outputs[i])
    print(wer)

print(f"WER: {wer:.2f}")


1.5
1.0
0.42857142857142855
0.6666666666666666
0.6363636363636364
1.125
1.0
1.1666666666666667
0.4
0.5384615384615384
1.0
0.5
0.5714285714285714
0.3333333333333333
0.4375
0.6153846153846154
0.4166666666666667
0.9166666666666666
0.6363636363636364
0.8
0.6363636363636364
0.36363636363636365
0.23529411764705882
0.8
0.4
0.5454545454545454
0.5
0.5
0.75
0.8333333333333334
0.5
0.5714285714285714
0.5
0.8571428571428571
0.5
0.4375
0.3333333333333333
0.6666666666666666
0.5
0.6666666666666666
0.4444444444444444
0.42857142857142855
0.2857142857142857
0.5
1.0
0.3333333333333333
0.25
0.5555555555555556
0.5
0.375
0.5
1.0
0.6666666666666666
0.5714285714285714
0.25
0.6
0.4444444444444444
0.5714285714285714
0.5
0.375
0.5
1.4
0.8571428571428571
0.8
0.5714285714285714
0.5
0.3333333333333333
0.625
0.625
0.6
0.6
0.75
0.6
0.6666666666666666
0.75
0.5714285714285714
0.5
0.3
0.4444444444444444
1.5
0.25
1.0
0.2857142857142857
0.6
0.25
0.42857142857142855
0.2857142857142857
0.42857142857142855
0.4
0.6
0.222222222

In [ ]:
PHONE_DEF = [
    'AA', 'AE', 'AH', 'AO', 'AW',
    'AY', 'B',  'CH', 'D', 'DH',
    'EH', 'ER', 'EY', 'F', 'G',
    'HH', 'IH', 'IY', 'JH', 'K',
    'L', 'M', 'N', 'NG', 'OW',
    'OY', 'P', 'R', 'S', 'SH',
    'T', 'TH', 'UH', 'UW', 'V',
    'W', 'Y', 'Z', 'ZH'
]
PHONE_DEF_SIL = PHONE_DEF + ['SIL']

In [ ]:
assert(False)

AssertionError: 

In [ ]:
nbest = utils.lm_decode(
    ngramDecoder,
    logits[0],
    blankPenalty=blank_penalty,
    returnNBest=False,
    rescore=False,
)

In [ ]:
print(nbest)

 would a bowl further in a man's height make him a putty all day c


In [ ]:
print(len(PHONE_DEF_SIL))

40


In [ ]:
for i in true_seqs[1]:
    print(PHONE_DEF_SIL[i - 1])

R
IH
CH
SIL
P
ER
CH
AH
S
T
SIL
S
EH
V
R
AH
L
SIL
S
AY
N
D
SIL
L
IH
TH
AH
G
R
AE
F
S
SIL


In [ ]:
print(true_seqs[0])

[32 18  1 20 28  3 29 18 40 28 18 20  3 23 29 17  9 12  9 40]


In [ ]:
out = []
for i in range(len(all_logits)):
    out.append(utils.lm_decode(all_logits[i]))
    print(out[i])

In [ ]:
print(out[4])

In [ ]:
total_edit_distance = 0
total_seq_length = 0

for i in range(len(out)):
    matcher = SequenceMatcher(
        a=trueSeq.tolist(), b=out[i].tolist()
    )
    decoded_seqs.append(decodedSeq)
    true_seqs.append(trueSeq)
    total_edit_distance += matcher.distance()
    total_seq_length += len(trueSeq)

avgDayLoss = np.sum(allLoss) / len(testLoader)
cer = total_edit_distance / total_seq_length

In [ ]:
# lmDir = base_dir+'/lm_5gram/speech_5gram/lang_test'
# ngramDecoder_5gram = utils.build_lm_decoder(
#     lmDir,
#     acoustic_scale=0.5,
#     nbest=100,
#     beam=18
# )

# MODEL_CACHE_DIR = '/home3/skaasyap/willett/lm_cache_weights'
# # Load OPT 6B model
# llm, llm_tokenizer = utils.build_opt(cacheDir=MODEL_CACHE_DIR,
#                                               device='auto',
#                                               load_in_8bit=True
# )

In [ ]:
/home3/skaasyap/willett/outputs/neurips_model_seed_0